In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_epochs = 50
batch_size = 40
learning_rate = 0.001
classes = ('plane', 'car' , 'bird',
    'cat', 'deer', 'dog',
    'frog', 'horse', 'ship', 'truck')

cuda


In [6]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)


Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset
    , batch_size = batch_size
    , shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset
    , batch_size = batch_size
    , shuffle = True)
n_total_step = len(train_loader)
print(n_total_step)


1250


In [8]:
model = models.vgg16(pretrained = True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)


In [9]:
for epoch in range(num_epochs):
    n_corrects = 0
    n_labels = 0
    for i, (imgs , labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        labels_hat = model(imgs)
        loss_value = criterion(labels_hat, labels)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        n_labels += labels.size(0)
        n_corrects += (labels_hat.argmax(axis=1)==labels).sum().item()
        
        if (i+1) % 250 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/n_labels):.2f}%')
            
    with torch.no_grad():
        number_corrects = 0
        number_samples = 0
        for i, (test_images_set , test_labels_set) in enumerate(test_loader):
            test_images_set = test_images_set.to(device)
            test_labels_set = test_labels_set.to(device)

            y_predicted = model(test_images_set)
            labels_predicted = y_predicted.argmax(axis = 1)
            number_corrects += (labels_predicted==test_labels_set).sum().item()
            number_samples += test_labels_set.size(0)
        print(f'Overall accuracy {(number_corrects / number_samples)*100}%')
        print()


epoch 1/50, step: 250/1250: loss = 0.53717, acc = 74.27%
epoch 1/50, step: 500/1250: loss = 0.55956, acc = 79.51%
epoch 1/50, step: 750/1250: loss = 0.28107, acc = 82.15%
epoch 1/50, step: 1000/1250: loss = 0.38966, acc = 83.80%
epoch 1/50, step: 1250/1250: loss = 0.25437, acc = 85.00%
Overall accuracy 89.92%

epoch 2/50, step: 250/1250: loss = 0.31646, acc = 92.68%
epoch 2/50, step: 500/1250: loss = 0.24950, acc = 92.33%
epoch 2/50, step: 750/1250: loss = 0.09811, acc = 92.54%
epoch 2/50, step: 1000/1250: loss = 0.24191, acc = 92.62%
epoch 2/50, step: 1250/1250: loss = 0.09769, acc = 92.77%
Overall accuracy 89.73%

epoch 3/50, step: 250/1250: loss = 0.12554, acc = 95.36%
epoch 3/50, step: 500/1250: loss = 0.09817, acc = 95.33%
epoch 3/50, step: 750/1250: loss = 0.15108, acc = 95.23%
epoch 3/50, step: 1000/1250: loss = 0.09392, acc = 95.26%
epoch 3/50, step: 1250/1250: loss = 0.07113, acc = 95.30%
Overall accuracy 91.97999999999999%

epoch 4/50, step: 250/1250: loss = 0.04867, acc = 97

Overall accuracy 92.99%

epoch 27/50, step: 250/1250: loss = 0.00305, acc = 99.96%
epoch 27/50, step: 500/1250: loss = 0.00007, acc = 99.97%
epoch 27/50, step: 750/1250: loss = 0.02011, acc = 99.92%
epoch 27/50, step: 1000/1250: loss = 0.00030, acc = 99.89%
epoch 27/50, step: 1250/1250: loss = 0.00064, acc = 99.91%
Overall accuracy 94.01%

epoch 28/50, step: 250/1250: loss = 0.00025, acc = 99.99%
epoch 28/50, step: 500/1250: loss = 0.00149, acc = 99.95%
epoch 28/50, step: 750/1250: loss = 0.00056, acc = 99.94%
epoch 28/50, step: 1000/1250: loss = 0.00004, acc = 99.94%
epoch 28/50, step: 1250/1250: loss = 0.00038, acc = 99.96%
Overall accuracy 94.01%

epoch 29/50, step: 250/1250: loss = 0.00042, acc = 99.99%
epoch 29/50, step: 500/1250: loss = 0.00055, acc = 100.00%
epoch 29/50, step: 750/1250: loss = 0.00009, acc = 100.00%
epoch 29/50, step: 1000/1250: loss = 0.00253, acc = 99.98%
epoch 29/50, step: 1250/1250: loss = 0.00009, acc = 99.97%
Overall accuracy 93.39%

epoch 30/50, step: 250

In [ ]:
model = torch.load("cifar-10.pth")

In [8]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(test_loader):
        test_images_set = test_images_set.to(device)
        test_labels_set = test_labels_set.to(device)
    
        y_predicted = model(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 93.07%


In [6]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(test_loader):
        test_images_set = test_images_set.to(device)
        test_labels_set = test_labels_set.to(device)
    
        y_predicted = model(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 92.74%


In [9]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(train_loader):
        test_images_set = test_images_set.to(device)
        test_labels_set = test_labels_set.to(device)
    
        y_predicted = model(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 98.934%


In [10]:
torch.save(model, "cifar-10_2.pth")


In [ ]:
uncorrect_dict = {
    "img_before": [],
    "img_after" : [],
    "softlabel_before" : [],
    "softlabel_after": []
}

# 또 틀린 애들
uncorrect_sp_dict = {
    "img_before": [],
    "img_after" : [],
    "softlabel_before" : [],
    "softlabel_after": []
}

# 틀렸다 맞은 애들
correct_dict = {
    "img_before": [],
    "img_after" : [],
    "softlabel_before" : [],
    "softlabel_after": []
}